
 
# **PROJETO AM_2019**

> Base de dados:  [Image segmentation](http://archive.ics.uci.edu/ml/datasets/image+segmentation); Tabela com 2100 objetos e 7 classes. cada objeto tem 19 variáveis onde as 9 primeiras sao o Shape View e as 10 ultimas RGB View 


* Test data: 2100; Training Data: 210
* 19 continuos attributes
* Classes:  brickface, sky, foliage, cement, window, path, grass.






## **1)**
1. Executar o algoritmo do artigo na tabela shape view e na tabela rbg view, 100 vezes p/ obter uma *partição fuzzy* em 7 grupos e selecione o melhor resultado; (Equação (27): Atualizar os *protótipos* dos grupos; Equação (31): Atualizar os pesos de relevância das variáveis; Equação (32): Atualizar o grau de *pertinência* dos objetos.)
2. A partir da partição fuzzy produza uma partição *crisp* em 7 grupos
3. Calcular índice de Rand corrigido em relação à partição à priori em 7 classes, para os resultados obtidos em cada view
4. Calcular o índice de Rand corrigido entre as partições obtidas com cada view



*  Obs: Para o melhor resultado imprimir: 
    1. a matrix de protótipos 
    2. o vetor de peso de relevância das variáveis. 
    3. a partição crisp (para cada grupo, a lista de objetos), 
    4. o numero de objetos de cada grupo crisp, 
    5. 0 indice de Rand corrigido.

## **2)**
1. Use *validação cruzada estratificada "30 times ten-fold* para avaliar e comparar os classificadores combinados. Se necessário, retirar do conjunto de aprendizagem um conjunto de validação para fazer ajuste de parametros.
2. Obter uma estimativa pontual e um intervalo de confiaça para a taxa de acerto de cada classificador.
3. Usar o *Wilcoxon signed-ranks test* (teste nao paramétrico) para comparar os classificadores.

* Classificador combinado pela regra da soma a partir do classificador bayesiano gaussiano;
*Classificador combinado pela regra da soma a partir do classificador bayesiano baseado em
k-vizinhos para fazer a classificação dos dados.


In [0]:
# import os
# from google.colab import drive

# drive.mount('/gdrive', force_remount=True)


In [0]:
path = "/gdrive/My Drive/10º Período/Aprendizagem/Aprendizagem de Máquina/Projeto 1 - Francisco/Projeto"

# Imports

In [0]:
from scipy.spatial.distance import pdist, squareform
import scipy as scip
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import math
import random
import numpy
import os

from multiprocessing import Pool

from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
from sklearn.metrics import adjusted_rand_score
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import Normalizer
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


# Carregar e inicalizar dados

Vamos carregar o dataset. Temos um dataset 'segmentation.data' com apenas 210 linhas e um 'segmentation.test' com 2100 linhas. Iremos usar o 'segmentation.test' no nosso modelo.

In [0]:
TRAIN_DATA_URL = "http://archive.ics.uci.edu/ml/machine-learning-databases/image/segmentation.data"
TEST_DATA_URL = "http://archive.ics.uci.edu/ml/machine-learning-databases/image/segmentation.test"


# Dados para treinamento a serem comparados
train_file_path = tf.keras.utils.get_file("segmentation.data.csv", TRAIN_DATA_URL) #210
# Dados de teste para classificação
test_file_path = tf.keras.utils.get_file("segmentation.test", TEST_DATA_URL) #2100

#Converter CSV em Dataframe
dataset = pd.read_csv(test_file_path, skiprows=[0,1,2])
dataset = dataset.dropna()
backup_dataset = dataset
# Names of the variables
dataset_names = list(dataset)
view_classes = backup_dataset.index

#-------Converter variáveis categóricas em numéricas---------------------#
# 0 = Brickface - 1 = Sky - 2 = Foliage 
# 3 = Cement - 4 = Window - 5 = Path - 6 = Grass
class_converted = LabelEncoder().fit_transform(view_classes)

352256/345084 [==============================] - 0s 0us/step


In [0]:
dataset

,REGION-CENTROID-COL,REGION-CENTROID-ROW,REGION-PIXEL-COUNT,SHORT-LINE-DENSITY-5,SHORT-LINE-DENSITY-2,VEDGE-MEAN,VEDGE-SD,HEDGE-MEAN,HEDGE-SD,INTENSITY-MEAN,RAWRED-MEAN,RAWBLUE-MEAN,RAWGREEN-MEAN,EXRED-MEAN,EXBLUE-MEAN,EXGREEN-MEAN,VALUE-MEAN,SATURATION-MEAN,HUE-MEAN
GRASS,110.0,189.0,9,0.000000,0.0,1.000000,0.666667,1.222222,1.186342,12.925926,10.888889,9.222222,18.666668,-6.111111,-11.111111,17.222221,18.666668,0.508139,1.910864
GRASS,86.0,187.0,9,0.000000,0.0,1.111111,0.720082,1.444444,0.750309,13.740741,11.666667,10.333334,19.222221,-6.222222,-10.222222,16.444445,19.222221,0.463329,1.941465
GRASS,225.0,244.0,9,0.000000,0.0,3.388889,2.195113,3.000000,1.520234,12.259259,10.333334,9.333334,17.111110,-5.777778,-8.777778,14.555555,17.111110,0.480149,1.987902
GRASS,47.0,232.0,9,0.000000,0.0,1.277778,1.254621,1.000000,0.894427,12.703704,11.000000,9.000000,18.111110,-5.111111,-11.111111,16.222221,18.111110,0.500966,1.875362
GRASS,97.0,186.0,9,0.000000,0.0,1.166667,0.691215,1.166667,1.005540,15.592592,13.888889,11.777778,21.111110,-5.111111,-11.444445,16.555555,21.111110,0.442661,1.863654
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CEMENT,32.0,158.0,9,0.000000,0.0,0.944445,0.862963,0.833333,0.611111,7.962963,6.333334,11.888889,5.666666,-4.888889,11.777778,-6.888889,11.888889,0.520578,-1.982834
CEMENT,8.0,162.0,9,0.111111,0.0,1.611111,2.062962,0.333333,0.133333,8.370370,6.666666,12.000000,6.444445,-5.111111,10.888889,-5.777778,12.000000,0.484805,-2.044946
CEMENT,128.0,161.0,9,0.000000,0.0,0.555555,0.251852,0.777778,0.162963,7.148148,5.555555,10.888889,5.000000,-4.777778,11.222222,-6.444445,10.888889,0.540918,-1.996307
CEMENT,150.0,158.0,9,0.000000,0.0,2.166667,1.633334,1.388889,0.418518,8.444445,7.000000,12.222222,6.111111,-4.333334,11.333333,-7.000000,12.222222,0.503086,-1.943449


In [0]:
dataset.describe()

,REGION-CENTROID-COL,REGION-CENTROID-ROW,REGION-PIXEL-COUNT,SHORT-LINE-DENSITY-5,SHORT-LINE-DENSITY-2,VEDGE-MEAN,VEDGE-SD,HEDGE-MEAN,HEDGE-SD,INTENSITY-MEAN,RAWRED-MEAN,RAWBLUE-MEAN,RAWGREEN-MEAN,EXRED-MEAN,EXBLUE-MEAN,EXGREEN-MEAN,VALUE-MEAN,SATURATION-MEAN,HUE-MEAN
count,2100.000000,2100.000000,2100.0,2100.000000,2100.000000,2100.000000,2100.000000,2100.000000,2.100000e+03,2100.000000,2100.000000,2100.000000,2100.000000,2100.000000,2100.000000,2100.000000,2100.000000,2100.000000,2100.000000
mean,124.940476,123.483333,9.0,0.014921,0.004550,1.890820,5.708299,2.406772,7.904224e+00,37.047654,32.806667,44.205556,34.130741,-12.722963,21.473704,-8.750741,45.162381,0.427259,-1.365147
std,72.858637,57.431428,0.0,0.041024,0.023573,2.649453,44.989359,3.469954,5.347107e+01,38.135291,34.994538,43.510119,36.303768,11.588214,19.654107,11.606996,42.900582,0.228458,1.544278
min,1.000000,11.000000,9.0,0.000000,0.000000,0.000000,0.000000,0.000000,-1.589457e-08,0.000000,0.000000,0.000000,0.000000,-49.666668,-12.444445,-33.888890,0.000000,0.000000,-3.044175
25%,62.000000,81.000000,9.0,0.000000,0.000000,0.722222,0.349603,0.833332,4.216377e-01,7.472222,7.000000,9.666667,6.222222,-18.583333,4.305556,-17.000000,11.777778,0.284934,-2.188539
50%,121.000000,122.000000,9.0,0.000000,0.000000,1.277776,0.833333,1.444444,9.897442e-01,21.666666,19.666668,27.777779,20.444445,-10.888889,19.666666,-11.000000,28.666666,0.375064,-2.052625
75%,188.250000,171.250000,9.0,0.000000,0.000000,2.222221,1.807406,2.555556,2.251852e+00,53.277778,47.333332,65.000000,46.388888,-4.222222,36.111110,-3.222222,65.000000,0.540228,-1.565745
max,254.000000,251.000000,9.0,0.333333,0.222222,29.222221,991.718400,44.722225,1.386329e+03,143.444440,137.111110,150.888890,142.555560,9.888889,82.000000,24.666666,150.888890,1.000000,2.912480


 
  Coluna: Region-Pixel-Count;
  
  Essa coluna é composta de valores constantes, seu desvio padrao é 0 e não 
  é capaz de contribuir de forma significativa para os resultados do algoritmo.
  Vamos remove-la.
  Após isso, vamos normalizar e dividir o dataset em Shape View e RGB View


In [0]:
#--------------- Dividir as views ----------------------#
#Shape View (2)
drop_index_shape =  [2, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
shape_view = dataset
shape_view = shape_view.drop(shape_view.columns[drop_index_shape], axis=1)
shape_names = list(shape_view)

#RGB View (3)
drop_index_rgb = [0, 1, 2, 3, 4, 5, 6, 7, 8]
rgb_view = dataset
rgb_view = rgb_view.drop(rgb_view.columns[drop_index_rgb], axis=1)
rgb_names = list(rgb_view)

#---------------- Normalizar as views ------------------#
scaler = Normalizer()
# Normalizer RGB
rgb_norm = scaler.fit_transform(rgb_view)
rgb_normalizer = pd.DataFrame(rgb_norm, columns=rgb_names)

# Normalizer shape
shape_norm = scaler.fit_transform(shape_view)
shape_normalizer = pd.DataFrame(shape_norm, columns=shape_names)

In [0]:
shape_normalizer.describe()

,REGION-CENTROID-COL,REGION-CENTROID-ROW,SHORT-LINE-DENSITY-5,SHORT-LINE-DENSITY-2,VEDGE-MEAN,VEDGE-SD,HEDGE-MEAN,HEDGE-SD
count,2100.000000,2100.000000,2100.000000,2100.000000,2100.000000,2100.000000,2100.000000,2.100000e+03
mean,0.631975,0.672090,0.000103,0.000024,0.011057,0.017963,0.013457,2.668077e-02
std,0.269237,0.248089,0.000366,0.000138,0.016146,0.072342,0.018572,8.809050e-02
min,0.002881,0.047381,0.000000,0.000000,0.000000,0.000000,0.000000,-1.047666e-10
25%,0.439622,0.510203,0.000000,0.000000,0.003876,0.001985,0.004391,2.342812e-03
50%,0.693619,0.710807,0.000000,0.000000,0.007174,0.004765,0.008444,5.866003e-03
75%,0.849139,0.888807,0.000000,0.000000,0.012604,0.010013,0.015938,1.323292e-02
max,0.998613,0.999862,0.006494,0.001562,0.215806,0.940037,0.276481,9.650939e-01


In [0]:
rgb_normalizer.describe()

,INTENSITY-MEAN,RAWRED-MEAN,RAWBLUE-MEAN,RAWGREEN-MEAN,EXRED-MEAN,EXBLUE-MEAN,EXGREEN-MEAN,VALUE-MEAN,SATURATION-MEAN,HUE-MEAN
count,2100.000000,2100.000000,2100.000000,2100.000000,2100.000000,2100.000000,2100.000000,2100.000000,2100.000000,2100.000000
mean,0.338294,0.287274,0.426804,0.300804,-0.153060,0.265531,-0.112471,0.451914,0.019788,-0.050977
std,0.099141,0.124667,0.095761,0.134180,0.149611,0.223872,0.215775,0.083632,0.035943,0.129567
min,0.000000,0.000000,0.000000,0.000000,-0.503916,-0.358232,-0.512190,0.000000,0.000000,-0.987420
25%,0.322039,0.246054,0.389035,0.215113,-0.240903,0.163712,-0.247217,0.453877,0.002082,-0.049238
50%,0.379016,0.338381,0.477204,0.339547,-0.146714,0.277000,-0.150054,0.481689,0.006623,-0.019154
75%,0.398399,0.367452,0.488203,0.392825,-0.103404,0.386025,-0.053222,0.488680,0.019340,-0.008404
max,0.441030,0.472173,0.490363,0.490093,0.483028,0.688844,0.536213,0.539349,0.272190,0.121548


# **Utilities**

Introduziremos uma função para reduzir a memória utilizada pelo nosso dataset ao escolher o menor tipo em tamanho de bytes capaz de armazenar todos os valores de uma determinada coluna.

# **Algoritmo**

1. Algoritmo **"Variable-wise kernel fuzzy clustering algorithm with kernelization of the
metric"** - page 10

Executar o algoritmo do artigo na tabela shape view e na tabela rbg view, 100 vezes p/ obter uma *partição fuzzy* em 7 grupos e selecione o melhor resultado; (Equação (27): Atualizar os *protótipos* dos grupos; Equação (31): Atualizar os pesos de relevância das variáveis; Equação (32): Atualizar o grau de *pertinência* dos objetos.)


In [0]:
#----------------------------------
# FUNCOES UPDATES E FUNCAO KERNEL
#----------------------------------

def distance_variance(x):
  n = len(x.index)
  p = len(x.columns)
  dist_variable_j = x.loc[0]
  variance_matrix = np.full((n,p),(dist_variable_j.quantile(0.1) + dist_variable_j.quantile(0.9))/2)

  for k in range(n):
    # Copia da linha(objeto) da matrix de dados.
    dist_row = x.loc[k]

    for j in range(p):
      for l in range(p):
        # Para cada variavel do objeto tiramos a distancia relativa quadradica
        # em relacao as outras variaveis do objeto.
        #dist_variable_j[l] = (dist_row[j] - dist_row[l])**2
        dist_variable_j[l] = np.power((dist_row[j] - dist_row[l]),2.0)
      # Adiciona na matrix o valor da media entre os quantiles 0.1 e 0.9
      # da distribuicao de distancia quadraticas das variaveis.
      #variance_matrix[k][j] = (dist_variable_j.quantile(0.1) + dist_variable_j.quantile(0.9))/2
      variance_matrix[k][j] = (np.quantile(np.array(dist_variable_j),0.1) + np.quantile(np.array(dist_variable_j),0.9))/2.0

  return variance_matrix
         
         
def gaussian_kernel(x_kj, v_ij, mean_variable):
  # Exponencial gaussiana dado uma variavel do objeto,
  # uma variavel do centroid(prototipo) e a variacia relativa
  # a variavel do objeto.
  return math.exp(-( (x_kj - v_ij)**2 ) / mean_variable)

def initialize_centroid(v,x):
  # Inicializa os prototipos aleariamente entre a distribuicao
  # de objetos x.
  index_used = np.array([])
  random_index = len(x.index)
  for i in range(np.ma.size(v,0)):
    random_index = random.randint(0,len(x.index)-1)

    if random_index in index_used :
      while True:
        random_index = random.randint(0,len(x.index)-1)
        if not(random_index in index_used):
          index_used = np.append(index_used,[random_index])
    else:
      index_used = np.append(index_used,[random_index]) 

    for j in range(np.ma.size(v,1)):
        v[i][j] = x.loc[random_index][j]
  return v

def calc_update_centroid(u_i, x, v_i, m, variance):
  cluster_v = v_i 
  for j in range(len(v_i)):
    p = numpy.float64(0.0) # numerador da razao
    q = numpy.float64(0.0) # denominador da razao
    for k in range(len(x.index)):
      # somatorio calculando a kernel gasussiano para cada objeto
      # dado uma variavel j
      p += np.multiply(np.power(u_i[k],m) , np.multiply(gaussian_kernel(x.loc[k][j],
                                                                        v_i[j], 
                                                                        variance[k][j]), 
                                                                        x.loc[k][j]))
      q += np.multiply(np.power(u_i[k], m) , gaussian_kernel(x.loc[k][j],
                                                             v_i[j], 
                                                             variance[k][j]))
    cluster_v[j] = numpy.float64(p/q)


  return cluster_v
      

def update_centroid(u, x, v, m, variance):
  args_calc = []
  for i in range(np.ma.size(v,0)):
    args_calc = args_calc + [tuple((u[i], dataset, v[i], m, variance))]
    
  new_v = []

  pl = Pool(os.cpu_count())
  v_matrix_reference = pl.starmap(calc_update_centroid,args_calc)
  for i in v_matrix_reference:
    new_v += [i]
  
  pl.close()
  return np.array(new_v)

def calc_weights(u, x_j, v_j, m, c, variance_j):
  # denominador da razao
  q = numpy.float64(0.0)
  for i in range(c):
    for k in range(len(x_j)):
      # Calcula o somatorio relativa a variavel j
      q += np.multiply(np.power(u[i][k],m), (2.0 - 2.0 * gaussian_kernel(x_j[k],
                                                                      v_j[i],
                                                                      variance_j[k])))
  return q

def update_weights(u, x, v, m, c, lambda_weights, variance):
  # numerador da fracao
  p = numpy.float64(1.0)
  # O numerador eh o mesmo para o calculo de todos os pesos
  for l in range(len(x.columns)):
    # variavel resultado do somatorio mais interno
    p_inner = numpy.float64(0.0)
    for i in range(c):
      for k in range(len(x.index)):
        p_inner += np.multiply(np.power(u[i][k],m),
                               (2.0 - 2.0 * gaussian_kernel(x.loc[k][l],v[i][l],
                                                        variance[k][l])))
    # Calcula o produtorio
    p = np.multiply(p,p_inner) 
  
  p = np.power(p,(1/len(x.columns)))
  
  args_calc = []
  data_array = np.array(x)
  for j in range(len(x.columns)):
    args_calc = args_calc + [tuple((u, data_array[:,j], v[:,j], m, c, variance[:,j]))]
    
  lambda_weights_p = np.full(len(x.columns), p)
  qs = []
  pl = Pool(os.cpu_count())
  v_matrix_reference = pl.starmap(calc_weights,args_calc)
  for i in v_matrix_reference:
    qs = np.append(qs,[1/i])

  lambda_weights = np.multiply(lambda_weights_p,qs)
  #print(new_v)
  pl.close()
  return lambda_weights

def dist_func(x_k, v_i, p, lambda_i, k, variance):
  # Calcula a funcao distancia quadradica de um objeto para um centroid(prototipo)
  # utilizando a funcao kernel gaussiana.
  phi = numpy.float64(0.0)
  for j in range(p):
    # Calculo relativo a cada variavel do objeto e centroid
    phi += lambda_i[j] * (2 - 2 * gaussian_kernel(x_k[j],v_i[j], variance[k][j]))
  
  return phi

def calc_update_membership(u_i, x, v, m, c, weights, variance, i):
  for k in range(len(x.index)):
    sum_e = numpy.float64(0.0)
    for h in range(c):
      # Calculo da razao entre a distancia do objeto k
      # e o prototipo i e a distancia do objeto k e o
      # prototipo h elevado a 1/(m-1)
      sum_e += np.power(np.multiply(dist_func(x.loc[k],v[i], len(x.columns), weights, k, variance), 1/(dist_func(x.loc[k],v[h], len(x.columns), weights, k, variance))),(1/(m-1)))
    # Calculo do grau de pertinencia
    u_i[k] = 1/sum_e

  return u_i

def update_membership(u, x, v, m, c, weights, variance):
  args_calc = []

  for i in range(c):
    args_calc = args_calc + [tuple((u[i], x, v, m, c, weights, variance, i))]
  
  new_u = []
  pl = Pool(os.cpu_count())
  u_matrix_reference = pl.starmap(calc_update_membership,args_calc)
  for i in u_matrix_reference:
    new_u += [i]
  pl.close()
  return np.array(new_u)
        
      
def calc_j(u, x, v, m, c, weights, variance):
  # Calculo do criterio de adequacao que mede
  # o ajuste entre grupos e seus centroids
  J = numpy.float64(0.0)
  for i in range(c):
    for k in range(len(x.index)):
      J += np.multiply(np.power(u[i][k],m) , dist_func(x.loc[k], v[i], len(x.columns), weights, k, variance))
  return J

def lim_J(J_new,J_old,epsilon_lim):
  #print(epsilon_lim)
  if abs(J_new - J_old) <= epsilon_lim:
    return False
  else:
    return True

#Calcula as distancias de cada objeto para os centróides dos c (7) clusters
def obj_distances(c, dataset, v, weights, variance): 
  n = len(dataset.index)
  p = len(dataset.columns)
  distances_matrix = np.zeros(shape=(n,c))
  for item in range(n):
    for c_idx in range(c):
      v_i = v[c_idx]
      x_k = dataset.loc[item]
      distances_matrix[item, c_idx] = dist_func(x_k, v_i, p, weights, item, variance)
  return distances_matrix

def generate_clusters(c, lambda_weights, matrix_pertinencia):
    elements_cluster_location = matrix_pertinencia.argmax(axis=1) #retorna a posição do maior argumento de cada linha, nesse caso, o maior grau de pertinencia, ou seja, o cluster para cada objeto
    n = len(elements_cluster_location)

    cluster_prototypes = []
    cluster_size = []

    for c_idx in range(c):
        cluster_elements = []
        elements = np.argwhere(elements_cluster_location == c_idx) #Retorna quais linhas (ou seja, quais objetos), pertecem ao cluster c_idx
        for e in elements:
            cluster_elements.append(e[0]) #adc o objeto a lista de elementos do cluster
            
        cluster_prototypes.append(cluster_elements[:])
        cluster_size.append(len(cluster_elements))
    return cluster_prototypes, cluster_size, elements_cluster_location


In [0]:
#--------------DATASET RGB NORMALIZER----------
dataset = rgb_normalizer #Descomente p/ usar o data RGB normalizado
log_name = 'rgb_view'
#--------------DATASET SHAPE NORMALIZE---------
# dataset = shape_normalizer #Descomente p/ usar o data SHAPE normalizado
# log_name = 'shape_view'


for partition in range(100):
  ## Algoritm Variable-wise kernel fuzzy c-means clustering with kernalization of metric
  #---------------------------------
  # Passo 1: 
  #--------------------------------
  #Number of clusters
  c = 7

  # Number of variables
  p = len(dataset.columns)

  #parametro m que controla o fuzziness de cada padrao
  m = 1.6

  #T = iteration limit
  T = 150
  n = len(dataset.index)

  #u_ik = fuzzy membership degree
  u = np.full((c,n),np.float64(1/c))
  #print(u)

  # lambda_j(each cluster) = weights of variables
  lambda_weights = np.full(p,np.float64(1))
  #print(lambda_weights)

  # inicializa prototipo
  v = np.full((c,p),numpy.float64(0.0))
  v = initialize_centroid(v,dataset)
  #print(v)

  #---------------------------------
  # Passo 2:
  #---------------------------------
  t=1
  var_matrix = distance_variance(dataset)
  #print(var_matrix)

  #---------------------------------
  J_new = calc_j(u, dataset, v, m, c, lambda_weights, var_matrix)
  J_old = numpy.float64(0.0)
  epsilon_lim = numpy.float64(10 ** (-10))
  J_func = np.array([J_new])
  print(J_new)
  #----------------------------------
  if math.isnan(J_new):
     print("\nJ_new is Nan\n")
  else:
    while lim_J(J_new,J_old,epsilon_lim) and t<T:
      #---------------------------------
      # Passo 3:
      #---------------------------------

      # Atualizar prototipo de cada cluster eq. 27
      v = update_centroid(u, dataset, v, m, var_matrix)
      #print("Passo 3 concluido")
      #--------------------------------
      # Passo 4:
      #--------------------------------

      # Atualizar os pesos das variaveis eq. 31
      lambda_weights = update_weights(u, dataset, v, m, c, lambda_weights, var_matrix)
      #print("Passo 4 concluido")
      #--------------------------------
      # Passo 5:
      #--------------------------------

      # Atualizar matrix de grau de pertinencia eq. 32
      u = update_membership(u, dataset, v, m, c, lambda_weights, var_matrix)
      #print("Passo 5 concluido")
      #-------------------------------
      # Passo 6:
      #-------------------------------
      t = t+1
      print ("t = ", t)
      J_old = J_new
      J_new = calc_j(u, dataset, v, m, c, lambda_weights, var_matrix)
      J_func = np.append(J_func,[J_new])
      #print ("J_old: ", J_old)
      print ("J_new: ", J_new)

  #----------------------------------------RESULTADOS----------------------------#
      
    # Multiplcacao dos pesos globais para cada variavel tem q ser 1
    mult = np.float64(1)
    for i in lambda_weights:
      mult = mult * i
    print("Produto dos pesos de relevância das variavéis: ",  mult)

    # A soma do grau de pertinencia de um abjeto a cada cluster deve ser 1 
    membership_degree = 0
    for i in range(np.ma.size(u,0)):
      membership_degree += u[i][0]
    print("Soma do grau de pertinencia dos objetos: ", membership_degree)
    print("Função Objetivo otimizada: " + str(J_new) + "\nNumero de Iterações = " + str(t))


    print("Matriz de protótipos: \n", v)
    print("\n Vetor de Relevância das variáveis: \n", lambda_weights)

    #distances_matriz = obj_distances(c, dataset, v, lambda_weights, var_matrix)
    cluster_prototype, clusters_size, elements_cluster_location = generate_clusters(c, lambda_weights, u.transpose())

    log_file = str(log_name) + "_results_partition_" + str(partition) + ".txt"
    arq = open(log_file, 'a') 
    arq.write("\n Produto dos pesos de relevância das variavéis: " + str(mult) + 
              "\n Soma do grau de pertinencia dos objetos: " + str(membership_degree) +
              "\n Função Objetivo otimizada: " + str(J_new) + 
              "\n Numero de Iteraçoes: " + str(t) +
              "\n Matriz de protótipos: \n" + str(v) +
              "\n Vetor de Relevância das variáveis: \n" + str(lambda_weights))
    arq.close()
    for c_idx in range(c):
      print("\nCluster "+ str(c_idx) + ':')
      print("Number of objects: ", clusters_size[c_idx])
      print("List of objects:\n", cluster_prototype[c_idx])
      arq = open(log_file, 'a') 
      arq.write("\nCluster "+ str(c_idx) + ':' + 
                "\n Number of objects: " + str(clusters_size[c_idx]) +
                "\nList of objects:\n" + str(cluster_prototype[c_idx]))
      arq.close()

    #Estudar melhor e usar outras métricas
    rand_idx = adjusted_rand_score(view_classes, elements_cluster_location)
    rand_idx_conv = adjusted_rand_score(class_converted, elements_cluster_location)

    print("\nRand view_classes: " + str(rand_idx) + "\nRand class_converted: " + str(rand_idx_conv))
    arq = open(log_file, 'a') 
    arq.write("\nRand class_converted: " + str(rand_idx_conv) +"\nRand view_classes: " + str(rand_idx) )
    arq.close()

    archive = open('All_J_funcs_result.txt', 'a')
    archive.write("Resultado [" + str(partition) + "]: " + str(J_new) + '\n')
    archive.close()
    print('\n\nThe End\n\n\n')

NameError: ignored

In [0]:
print(rgb_normalizer.isna().sum())
rgb_normalizer.describe()

# **Resultados**